In [1]:
import tensorflow as tf
# import cv2 as cv
# import numpy as np
# import utils.model_helper as mh
# import utils.data_utils as du
import utils.metrics as M
import tensorflow.keras.backend as K
# import os
# import matplotlib.pyplot as plt
# from PIL import Image

2023-06-14 01:16:34.021296: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 01:16:34.068299: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 01:16:34.068770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 01:16:35.029406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
tf.__version__

In [ ]:
DATASET_PATH = '/home/irizqy/ml_ws/bangkit-ws/data/bizz.it-sim_dataset'

In [ ]:
(train_pairs, train_labels), (test_pairs, test_labels) = du.make_train_test_pairs(DATASET_PATH, .2, use_gray=False)
train_pairs.shape, train_labels.shape, test_pairs.shape, test_labels.shape

In [ ]:
np.unique(train_labels, return_counts=True), np.unique(test_labels, return_counts=True)

In [ ]:
w = 20
h = 15
fig = plt.figure(figsize=(8, 8))
columns = 8
rows = 8
for index, i in enumerate(range(1, (columns*rows +1)//2 + 1)):
    img = test_pairs[index][0]
    fig.add_subplot(rows, columns, 2*i - 1) 
    plt.title(test_labels[index])
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    img = test_pairs[index][1]
    fig.add_subplot(rows, columns, 2*i)
    plt.imshow(img, cmap='gray')
    plt.axis('off')


In [ ]:
IMG_SHAPE = (150, 150, 3)
BATCH_SIZE = 32
EPOCHS = 30

In [ ]:
train_step = train_pairs.shape[0] // BATCH_SIZE
test_step = test_pairs.shape[0] // BATCH_SIZE

print(train_step, test_step)

In [ ]:
class SiameseModel:

    def __init__(self, input_shape, embedding_dim=224):
        self.input_shape = input_shape
        self.embedding_dim = embedding_dim

    def _build(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=self.input_shape),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu',),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu',),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu',),
            tf.keras.layers.MaxPooling2D(),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.embedding_dim),
            tf.keras.layers.Dropout(.2),
        ])

        return model

In [ ]:
# sm = SiameseModel(IMG_SHAPE)

# featureExtractor = sm._build()

base_cnn =  tf.keras.applications.resnet.ResNet50(
    weights='imagenet', input_shape=IMG_SHAPE, include_top=False
)

# base_cnn.summary()

cnn_model = base_cnn.get_layer('conv5_block3_2_conv')

flatten = tf.keras.layers.GlobalAveragePooling2D()(cnn_model.output)
dense_2 = tf.keras.layers.Dense(units=224)(flatten)

featureExtractor = tf.keras.Model(base_cnn.input, dense_2)

for layer in featureExtractor.layers[:-25]:
    layer.trainable = False

featureExtractor.summary()

In [ ]:
# configure the siamese network
print("[INFO] building siamese network...")
imgA = tf.keras.layers.Input(shape=IMG_SHAPE)
imgB = tf.keras.layers.Input(shape=IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

In [ ]:
# finally, construct the siamese network
distance = tf.keras.layers.Lambda(lambda vectors: K.sqrt(K.maximum(K.sum(K.square(vectors[0] - vectors[1]), axis=1, keepdims=True), K.epsilon())))([featsA, featsB])
# distance = tf.keras.layers.Lambda(M.euclidean_distance)([featsA, featsB])
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
model = tf.keras.Model(inputs=[imgA, imgB], outputs=outputs)

In [ ]:
model.summary()

In [ ]:
tb = tf.keras.callbacks.TensorBoard(log_dir='./log/', histogram_freq=0,  
          write_graph=True, write_images=True)

In [ ]:
tf.keras.backend.clear_session()
# compile the model
print("[INFO] compiling model...")
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(),
	metrics=["accuracy"])
# train the model
print("[INFO] training model...")
history = model.fit(
	[train_pairs[:, 0], train_pairs[:, 1]], train_labels[:],
	validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels[:]),
	batch_size=BATCH_SIZE,
    callbacks=[tb],
	epochs=EPOCHS) 

In [ ]:
def plot_training(H):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")

In [ ]:
plot_training(history)

In [ ]:
model.save('/home/irizqy/ml_ws/bangkit-ws/src/logo-detector/im_similar')

In [2]:
model = tf.keras.models.load_model('/home/irizqy/ml_ws/bangkit-ws/src/logo-detector/im_similar')

2023-06-14 01:16:41.577103: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-14 01:16:41.577561: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
path_im1_1 = '/home/irizqy/ml_ws/bangkit-ws/src/logo-detector/cropped-logo0.jpg'
path_im1_2 = '/home/irizqy/Downloads/franchise/ct.png'

path_im2_1 = '/home/irizqy/Downloads/franchise/ct.png'
path_im2_2 = '/home/irizqy/Downloads/sabana_10.png'

In [5]:
im1 = mh.adjust_im(path_im2_2, (150, 150))
im2 = mh.adjust_im(path_im2_2, (150, 150))

model((im1, im2)).numpy()

array([[0.9861888]], dtype=float32)

In [ ]:
preds = []

BASE_TEST_PATH = '/home/irizqy/Downloads/franchise'
test_data = os.listdir(BASE_TEST_PATH)

im1 = mh.adjust_im(path_im2_2, (150, 150), use_gray=True)
# im2 = mh.adjust_im(path_im2_2, (150, 150))

# model.predict((im1, im2))[0][0]

for file in test_data:
    im2 = mh.adjust_im(os.path.join(BASE_TEST_PATH, file), (150, 150), use_gray=True)
    pred = model.predict((im1, im2))[0][0]
    preds.append(np.round(pred, 5))

preds = np.asarray(preds)

In [ ]:
max_idx = np.argmax(preds)
print(max_idx, preds[max_idx])

In [ ]:
{key: val.split('.')[0] for key, val in enumerate(test_data)}

In [ ]:
preds

In [ ]:
test_data[max_idx]